# Mock-up exam

In this mock-up exam, we will be using one of the classification tasks found in OpenML. More precisely, the task [*diabetes*](https://www.openml.org/search?type=data&id=37) (data_id=37) is selected. This task consists in the classification of patients into diabetes tested negative (class label = 0) or positive (class label = 1) according to the following eight features:
<ol>
<li>Number of times pregnant</li>
<li>Plasma glucose concentration a 2 hours in an oral glucose tolerance test</li>
<li>Diastolic blood pressure (mm Hg)</li>
<li>Triceps skin fold thickness (mm)</li>
<li>2-Hour serum insulin (mu U/ml)</li>
<li>Body mass index (weight in kg/(height in m)^2)</li>
<li>Diabetes pedigree function</li>
<li>Age (years)</li>
</ol>

Below you can find a baseline result achieved with default parameters.

In [35]:
import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

data_id = 37
test_size = 0.1
X, y = fetch_openml(data_id=data_id, return_X_y=True, as_frame=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=23)
clf = LogisticRegression(random_state=23).fit(X_train, y_train)
print(f'Test error: {(1 - accuracy_score(y_test, clf.predict(X_test)))*100:4.1f}%')

Test error: 19.5%


### Exercise 
Applying the logistic regression classifier, assess the effect of its parameters studied in the lab sessions on the classification error rate provided a train-test partition devoting 90%-10%, respectively. Use *random_state=23* to define this partition. In this process, try to improve the baseline result provided above. According to the results you have obtained, could you claim that this task is linearly separable? why? 

### Solution

**Experiments:** $\;$ exploring initial parameter values for solver, tolerance, C and maximum number of iterations. 

In [36]:
print('   solver     tol       C max_iter   etr   ete')
print('--------- ------- ------- -------- ----- -----')
for solver in ['lbfgs', 'newton-cg', 'sag', 'saga']:
    for tol in (1e-4, 1e-2, 1, 1e2, 1e4):
        for C in (1e-2, 1e-1, 1, 1e1, 1e2):
            for max_iter in (50, 100, 200, 500):
                clf = LogisticRegression(solver=solver, tol=tol, C=C, random_state=23, max_iter=max_iter).fit(X_train, y_train)
                etr = 1 - accuracy_score(y_train, clf.predict(X_train))
                ete = 1 - accuracy_score(y_test, clf.predict(X_test))
                print(f'{solver:>9} {tol:.1e} {C:.1e} {max_iter:8d} {etr:.1%} {ete:.1%}')

   solver     tol       C max_iter   etr   ete
--------- ------- ------- -------- ----- -----
    lbfgs 1.0e-04 1.0e-02       50 31.0% 29.9%
    lbfgs 1.0e-04 1.0e-02      100 24.2% 23.4%
    lbfgs 1.0e-04 1.0e-02      200 23.0% 18.2%
    lbfgs 1.0e-04 1.0e-02      500 23.0% 18.2%
    lbfgs 1.0e-04 1.0e-01       50 29.4% 24.7%
    lbfgs 1.0e-04 1.0e-01      100 23.2% 19.5%
    lbfgs 1.0e-04 1.0e-01      200 23.0% 19.5%
    lbfgs 1.0e-04 1.0e-01      500 23.0% 19.5%
    lbfgs 1.0e-04 1.0e+00       50 29.8% 23.4%
    lbfgs 1.0e-04 1.0e+00      100 22.0% 19.5%
    lbfgs 1.0e-04 1.0e+00      200 21.9% 18.2%
    lbfgs 1.0e-04 1.0e+00      500 21.9% 18.2%
    lbfgs 1.0e-04 1.0e+01       50 29.5% 32.5%
    lbfgs 1.0e-04 1.0e+01      100 22.9% 18.2%
    lbfgs 1.0e-04 1.0e+01      200 21.7% 18.2%
    lbfgs 1.0e-04 1.0e+01      500 21.7% 18.2%
    lbfgs 1.0e-04 1.0e+02       50 30.2% 35.1%
    lbfgs 1.0e-04 1.0e+02      100 22.4% 19.5%
    lbfgs 1.0e-04 1.0e+02      200 21.7% 18.2%
    lbfgs 1.0

**Regularization:** $\;$ parameter `C` (positive, $1.0$ by default) de-regularizes the training criterion
* **Possibility of under-adjustment:** $\;$ with a value close to zero (maximum regularization)
* **Possibility of overfitting:** $\;$ with a very high positive value (minimum regularization)

**Early stopping:** $\;$ saving computation and avoiding over-training ("regularize") by finishing earlier (in a few iterations)

In [14]:
for max_iter in (10, 20, 50, 100):
    clf = LogisticRegression(random_state=23, max_iter=max_iter).fit(X_train, y_train)
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))
    print(f"Test error with max_iter {max_iter}: {err_test:.1%}")

Test error with max_iter 10: 0.0%
Test error with max_iter 20: 3.3%
Test error with max_iter 50: 0.0%
Test error with max_iter 100: 0.0%
